In [22]:
#  This file contains utility functions for the project.
import numpy as np
import pandas as pd
import yfinance as yf

def fetch_stock_data(tickers, start_date=None, end_date=None, interval='1d'):
    """
    Fetches historical stock data for given tickers using yfinance.

    Parameters:
    - tickers (list of str): List of stock ticker symbols.
    - start_date (str): Start date for the data in format 'YYYY-MM-DD' (optional).
    - end_date (str): End date for the data in format 'YYYY-MM-DD' (optional).
    - interval (str): Data interval. Valid intervals: '1d', '1wk', '1mo', etc.

    Returns:
    - dict: A dictionary with tickers as keys and DataFrames as values.
    """
    stock_data = {}
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        data = stock.history(start=start_date, end=end_date, interval=interval)
        stock_data[ticker] = data

    return stock_data


# Compute additional metrics
def compute_metrics(data):
    """
    Computes additional metrics for stock data.

    Parameters:
    - data (dict): A dictionary with tickers as keys and DataFrames as values.

    Returns:
    - dict: A dictionary with tickers as keys and DataFrames as values.
    """
    metrics = {}
    for ticker, df in data.items():
        # Calculate daily return
        df['Daily Return'] = df['Close'].pct_change().fillna(0)

        # Calculate cumulative return
        df['Cumulative Return'] = (1 + df['Daily Return']).cumprod().fillna(1)

        # Calculate simple moving averages for specified periods
        df['SMA_3'] = df['Close'].rolling(window=3).mean()
        df['SMA_7'] = df['Close'].rolling(window=7).mean()
        df['SMA_30'] = df['Close'].rolling(window=30).mean()  # Approximating 1 month
        df['SMA_90'] = df['Close'].rolling(window=90).mean()  # Approximating 3 months

        # Calculate VWAP (Volume Weighted Average Price)
        if 'Volume' in df.columns:
            df['VWAP'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()
            df['VWAP_3'] = (df['Close'] * df['Volume']).rolling(window=3).sum() / df['Volume'].rolling(window=3).sum()
            df['VWAP_7'] = (df['Close'] * df['Volume']).rolling(window=7).sum() / df['Volume'].rolling(window=7).sum()
            df['VWAP_30'] = (df['Close'] * df['Volume']).rolling(window=30).sum() / df['Volume'].rolling(window=30).sum()
            df['VWAP_90'] = (df['Close'] * df['Volume']).rolling(window=90).sum() / df['Volume'].rolling(window=90).sum()

        metrics[ticker] = df

    return metrics

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dropout, Dense, Input
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm

sns.set()

# Configure Modeling Parameters and Fetch Data

Enter a ticker and date range you would like to build the model on.  This model takes a a single ticker's data.  Also enter a training size for the proportion of the data you want to include in your training set vs. your test set.

In [29]:
# stock configs
ticker = ['TSLA']
start_date = '2015-04-01'
end_date = '2024-04-05'

# model configs
train_size = 0.8

n_future = 1   # Number of days we want to look into the future based on the past days.
n_past = 30  # Number of past days we want to use to predict the future.

In [30]:
# Data Fetching
data = fetch_stock_data(ticker, start_date, end_date)[ticker[0]]
data.reset_index(drop=False, inplace=True)
data['Date'] = pd.to_datetime(data['Date']).dt.tz_localize(None)

print(data.shape)
included_days = len(data)
data.head()

(2268, 8)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2015-04-01,12.580000,12.820000,12.403333,12.506000,56919000,0.0,0.0
1,2015-04-02,12.682000,12.882000,12.666667,12.733333,75156000,0.0,0.0
2,2015-04-06,13.200000,13.850000,13.166667,13.540000,186837000,0.0,0.0
3,2015-04-07,13.500667,13.670667,13.409333,13.550000,65218500,0.0,0.0
4,2015-04-08,13.880000,14.060000,13.724667,13.844667,94546500,0.0,0.0


# Model Implementation

Below we implement the model, first we separate the data into a train-test split.  For this LSTM model, we will leverage continuous windows of time for training and test data.


In [31]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['Open', 'High', 'Low', 'Close', 'Volume']])

# Create input sequences
lookback = 30
X, y = [], []
for i in range(lookback, len(scaled_data)):
    X.append(scaled_data[i-lookback:i])
    y.append(scaled_data[i, 3])  # Closing price is at index 3
X, y = np.array(X), np.array(y)
# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [27]:
# Build the LSTM model
model = Sequential()
model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))  # Add an explicit Input layer
model.add(LSTM(units=50, return_sequences=True))              # No need to include input_shape here
model.add(Dropout(0.2))
model.add(LSTM(units=50))                                     # Second LSTM layer
model.add(Dropout(0.2))
model.add(Dense(units=1))                                     # Output layer
model.compile(optimizer='adam', loss='mean_squared_error')    # Compile the model
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
56/56 [==============================] - 7s 29ms/step - loss: 0.0084
Epoch 2/10
56/56 [==============================] - 2s 29ms/step - loss: 0.0024
Epoch 3/10
56/56 [==============================] - 2s 30ms/step - loss: 0.0020
Epoch 4/10
56/56 [==============================] - 2s 31ms/step - loss: 0.0018
Epoch 5/10
56/56 [==============================] - 2s 31ms/step - loss: 0.0018
Epoch 6/10
56/56 [==============================] - 3s 48ms/step - loss: 0.0015
Epoch 7/10
56/56 [==============================] - 2s 37ms/step - loss: 0.0015
Epoch 8/10
56/56 [==============================] - 2s 31ms/step - loss: 0.0016
Epoch 9/10
56/56 [==============================] - 2s 31ms/step - loss: 0.0018
Epoch 10/10
56/56 [==============================] - 2s 29ms/step - loss: 0.0014


In [ ]:
# Specify the number of iterations
num_iterations = 5

# Create traces for actual and predicted stock prices
trace_actual = go.Scatter(
    x=data['Date'][train_size+lookback:],
    y=data['Close'][train_size+lookback:],
    mode='lines',
    name='Actual Price',
    line=dict(color='blue')
)

traces_predicted = []
for i in range(num_iterations):
    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32)

    # Make predictions on the test data
    y_pred = model.predict(X_test)

    # Inverse transform the scaled data
    y_pred_inverse = scaler.inverse_transform(np.column_stack((np.zeros_like(y_pred), np.zeros_like(y_pred), np.zeros_like(y_pred), y_pred, np.zeros_like(y_pred))))[:, 3]

    # Create a trace for each iteration
    trace_predicted = go.Scatter(
        x=data['Date'][train_size+lookback:],
        y=y_pred_inverse,
        mode='lines',
        name=f'Predicted Price (Iteration {i+1})',
        line=dict(color=f'rgba(255, 0, 0, {0.2 + i*0.2})', dash='dot')
    )
    traces_predicted.append(trace_predicted)


In [33]:
# Create the layout
layout = go.Layout(
    title='Stock Price Prediction (Multiple Iterations)',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Price'),
    template='plotly_dark'
)

# Create the figure and display it
fig = go.Figure(data=[trace_actual] + traces_predicted, layout=layout)
fig.show()